### Importing libraries

In [112]:
import numpy as np
import pandas as pd
import random
import math

from sklearn import preprocessing # normalization library


### Robot Data
Pioneer 3dx

In [113]:
LMr=381e-3
R=195e-3

v_max= 1
v_min=0.3

Lt_max=1.2
Lt_min=0.3

Mean Velocity

In [114]:
v_mean=(v_max+v_min)/2
print(v_mean)

0.65


### Obstacle circle 

Sensor range data: $[dmin_1\quad dmax_1]\quad m$ 

Heading angle data: $[-\pi\quad\pi]\quad rad$ 


In [115]:
min_d1=0.3
max_d1=1

In [116]:
# Sensor Data gen 
Obs_Data=[round(random.uniform(min_d1, max_d1),4) for _ in range(3000*3)]
Obs_Data_M=np.array(Obs_Data).reshape(3000,3)

print('Raw data :',Obs_Data_M[:2,:])

normalizer=preprocessing.MinMaxScaler()
Obs_Data_Norm=normalizer.fit_transform(Obs_Data_M)
print('Normalized data :',Obs_Data_Norm[:2,:])


Raw data : [[0.6867 0.5531 0.7778]
 [0.5367 0.9258 0.9522]]
Normalized data : [[0.55285367 0.36172645 0.68244209]
 [0.33829209 0.89438331 0.93179868]]


In [117]:
# HA data gen

HA_Data=[round(random.uniform(-math.pi, math.pi),4) for _ in range(3000)]
HA_Data=np.array(HA_Data).reshape(3000,1)
print('Angle data :',HA_Data[:2])

Angle data : [[-2.2442]
 [-1.3575]]


Stack Normal Data

In [118]:


C1_Normal_Data=np.hstack((Obs_Data_Norm,HA_Data))
print('Normal array :',C1_Normal_Data[:2])

Normal array : [[ 0.55285367  0.36172645  0.68244209 -2.2442    ]
 [ 0.33829209  0.89438331  0.93179868 -1.3575    ]]


### Special Data for small angles and near obstacle

In [119]:
Obs_Data_Near=[round(random.uniform(min_d1,max_d1-0.6),4) for _ in range(1000*3)]
Obs_Data_M_Near=np.array(Obs_Data_Near).reshape(1000,3)

print('Raw data :',Obs_Data_M_Near[:2,:])


Norm_Near=normalizer.fit(Obs_Data_M)
Obs_Data_Near=Norm_Near.transform(Obs_Data_M_Near)

print('Normalized data :',Obs_Data_Near[:2,:])


Raw data : [[0.3278 0.373  0.3096]
 [0.3061 0.3992 0.3099]]
Normalized data : [[0.03947933 0.10433043 0.01301115]
 [0.00843942 0.14177505 0.01344009]]


Small angles

In [120]:
Small_Theta_Data=[round(random.uniform(-0.5, 0.5),4) for _ in range(1000)]
Small_Theta_Array=np.array(Small_Theta_Data).reshape(1000,1)
print('Angle data :',Small_Theta_Array[:2])

Angle data : [[ 0.1141]
 [-0.2252]]


Stack small

In [121]:
C1_Small_Data=np.hstack((Obs_Data_Near,Small_Theta_Array))
print('Small array :',C1_Small_Data[:2])

Small array : [[ 0.03947933  0.10433043  0.01301115  0.1141    ]
 [ 0.00843942  0.14177505  0.01344009 -0.2252    ]]


Stack all and Randomize

In [122]:
# Stack All

St_train_C1=np.vstack((C1_Normal_Data,C1_Small_Data))
np.random.shuffle(St_train_C1)
print(St_train_C1[:2])



[[0.92261479 0.19008146 0.41149557 0.738     ]
 [0.01344586 0.08274975 0.12539319 0.311     ]]


Make dataframe

In [123]:
Df_C1=pd.DataFrame(St_train_C1, columns=['LS','FS','RS','Thetad'])
Df_C1.sample(6)

,LS,FS,RS,Thetad
238,0.475182,0.009861,0.607235,0.7536
2836,0.727221,0.607546,0.400772,1.0085
35,0.112430,0.368015,0.914069,2.6392
236,0.564154,0.220666,0.335573,2.0572
3559,0.253898,0.894812,0.358879,3.0555
1092,0.780718,0.334000,0.660852,1.2558


### Inverse Kinematic Model

$$\omega_{R/L}=\left( 1\pm\frac{L_{MR}}{2L_T}\right)\frac{v}{R}$$

Gain near extremes normalized

In [124]:
def gain_near_extremes_normalized(x):
     return 4 * x * (1 - x)

In [125]:
def gain_near_zero(x, scale=10):
     gain = np.exp(-scale * x)
     normalized_gain = (gain - np.exp(-scale)) / (1 - np.exp(-scale))
     return normalized_gain
    

First Case: Left sensor nearest, objective angle +

In [126]:
odp=1
theta_d=3


S=-1
factor=round(gain_near_extremes_normalized(odp),4)
print('Factor : ',factor)
Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
print('Ltp: ',Lt_p)
V_p=round(v_max-(factor*(v_mean/2)),4)
print('vel: ',V_p)
if odp<0.2 and theta_d<0.5:
    fth=round(gain_near_zero(theta_d,scale=10),4)
    print('Factor theta:',fth)
    theta_d=round(fth*math.pi,4)
else:
    theta_d=theta_d
print('Tetha sel: ',theta_d)
W_R=round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)
W_L=round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)
print('Right Wheel vel: ',W_R)
print('Left Wheel vel: ',W_L)


def Case_1_R(odp,theta_d):
    if theta_d==0:
       theta_d=0.001
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d<0.5:
        fth=round(gain_near_zero(theta_d,scale=10),4)
        theta_d=round(fth*math.pi,4)
    else:
        theta_d=theta_d
    return round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)

    
def Case_1_L(odp,theta_d):
    if theta_d==0:
       theta_d=0.001
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d<0.5:
        fth=round(gain_near_zero(theta_d,scale=10),4)
        theta_d=round(fth*math.pi,4)
    else:
        theta_d=theta_d
    return round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)

    

Factor :  0
Ltp:  1.2
vel:  1.0
Tetha sel:  3
Right Wheel vel:  2.6859
Left Wheel vel:  7.5705


Second Case: Left sensor nearest, objective angle -

In [127]:

odp=0.6
theta_d=0
if theta_d==0:
    theta_d=-0.001
S=-1
factor=round(gain_near_extremes_normalized(odp),4)
print('Factor : ',factor)

Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
print('Ltp: ',Lt_p)
V_p=round(v_max-(factor*(v_mean/2)),4)
print('vel: ',V_p)

if odp<0.2 and theta_d>-0.5 and theta_d<0:
    fth=round(gain_near_zero(-1*theta_d),4)
    print('Factor theta:',fth)
    theta_d=fth*-math.pi
else:
    theta_d=theta_d
    
cf=math.pi/4
    
print('Tetha sel: ',theta_d)
W_R=round((1+((LMr*(theta_d+cf)*S)/(2*Lt_p)))*(V_p/R),4)
W_L=round((1-((LMr*(theta_d+cf)*S)/(2*Lt_p)))*(V_p/R),4)
print('Right Wheel vel: ',W_R)
print('Left Wheel vel: ',W_L)

def Case_2_R(odp,theta_d):
    if theta_d==0:
       theta_d=-0.001
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d>-0.5 and theta_d<0:
        fth=round(gain_near_zero(-1*theta_d),4)
        theta_d=fth*-math.pi
    else:
        theta_d=theta_d   
    cf=math.pi/4   
    return round((1+((LMr*(theta_d+cf)*S)/(2*Lt_p)))*(V_p/R),4)
    
def Case_2_L(odp,theta_d):
    if theta_d==0:
       theta_d=-0.001
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d>-0.5 and theta_d<0:
        fth=round(gain_near_zero(-1*theta_d),4)
        theta_d=fth*-math.pi
    else:
        theta_d=theta_d   
    cf=math.pi/4   
    return round((1-((LMr*(theta_d+cf)*S)/(2*Lt_p)))*(V_p/R),4)

Factor :  0.96
Ltp:  0.84
vel:  0.688
Tetha sel:  -0.001
Right Wheel vel:  2.9006
Left Wheel vel:  4.1558


Third case: Right sensor nearest, objective angle +

In [128]:
odp=0.3
theta_d=0.06
if theta_d==0:
    theta_d=0.001
S=-1
factor=round(gain_near_extremes_normalized(odp),4)
print('Factor : ',factor)

Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
print('Ltp: ',Lt_p)
V_p=round(v_max-(factor*(v_mean/2)),4)
print('vel: ',V_p)

if odp<0.2 and theta_d<0.5 and theta_d>0:
    fth=round(gain_near_zero(theta_d),4)
    print('Factor theta:',fth)
    theta_d=round(fth*math.pi)
else:
    theta_d=theta_d
cf=-math.pi/4
    
print('Tetha sel: ',theta_d)
W_R=round((1+((LMr*(theta_d+cf)*S)/(2*Lt_p)))*(V_p/R),4)
W_L=round((1-((LMr*(theta_d+cf)*S)/(2*Lt_p)))*(V_p/R),4)
print('Right Wheel vel: ',W_R)
print('Left Wheel vel: ',W_L)

def Case_3_R(odp,theta_d):
    if theta_d==0:
       theta_d=0.001
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d<0.5 and theta_d>0:
        fth=round(gain_near_zero(theta_d),4)
        theta_d=round(fth*math.pi)
    else:
        theta_d=theta_d
    cf=-math.pi/4
    return round((1+((LMr*(theta_d+cf)*S)/(2*Lt_p)))*(V_p/R),4)

def Case_3_L(odp,theta_d):
    if theta_d==0:
       theta_d=0.001
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d<0.5 and theta_d>0:
        fth=round(gain_near_zero(theta_d),4)
        theta_d=round(fth*math.pi)
    else:
        theta_d=theta_d
    cf=-math.pi/4
    return round((1-((LMr*(theta_d+cf)*S)/(2*Lt_p)))*(V_p/R),4)

Factor :  0.84
Ltp:  0.57
vel:  0.727
Tetha sel:  0.06
Right Wheel vel:  4.6321
Left Wheel vel:  2.8244


Fourth case: Right sensor nearest, objective angle -

In [129]:
odp=1
theta_d=-math.pi/2
if theta_d==0:
       theta_d=-0.001
S=-1

factor=round(gain_near_extremes_normalized(odp),4)

Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
print('Ltp: ',Lt_p)
V_p=round(v_max-(factor*(v_mean/2)),4)
print('vel: ',V_p)

if odp<0.2 and theta_d>-0.5 and theta_d<0:
    fth=round(gain_near_zero(-1*theta_d),4)
    print('Factor theta:',fth)
    theta_d=round(fth*-math.pi,4)
else:
    theta_d=theta_d


print('Tetha sel: ',theta_d)

W_R=round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)
W_L=round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)
print('Right Wheel vel: ',W_R)
print('Left Wheel vel: ',W_L)

def Case_4_R(odp,theta_d):
    if theta_d==0:
       theta_d=-0.001
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d>-0.5 and theta_d<0:
        fth=round(gain_near_zero(-1*theta_d),4)
        theta_d=round(fth*-math.pi,4)
    else:
        theta_d=theta_d
        
    return round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)


def Case_4_L(odp,theta_d):
    if theta_d==0:
       theta_d=-0.001
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d>-0.5 and theta_d<0:
        fth=round(gain_near_zero(-1*theta_d),4)
        theta_d=round(fth*-math.pi,4)
    else:
        theta_d=theta_d
        
    return round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)

Ltp:  1.2
vel:  1.0
Tetha sel:  -1.5707963267948966
Right Wheel vel:  6.407
Left Wheel vel:  3.8494


Fifth case: Front sensor nearest, objective angle +

In [130]:
odp=0.3
theta_d=3

if theta_d==0:
    theta_d=0.001
    
S=-1

factor=round(gain_near_extremes_normalized(odp),4)
print('Factor : ',factor)


Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
print('Ltp: ',Lt_p)
V_p=round(v_max-(factor*(v_mean/2)),4)
print('vel: ',V_p)

if odp<0.2 and theta_d<0.5:
    fth=round(gain_near_zero(theta_d,scale=10),4)
    print('Factor theta:',fth)
    theta_d=round(fth*math.pi,4)
else:
    theta_d=theta_d

print('Tetha sel: ',theta_d)
W_R=round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)
W_L=round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)
print('Right Wheel vel: ',W_R)
print('Left Wheel vel: ',W_L)


def Case_5_R(odp,theta_d):
    if theta_d==0:
        theta_d=0.001
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d<0.5:
        fth=round(gain_near_zero(theta_d,scale=10),4)
        theta_d=round(fth*math.pi,4)
    else:
        theta_d=theta_d
    return round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)


def Case_5_L(odp,theta_d):
    if theta_d==0:
        theta_d=0.001
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d<0.5:
        fth=round(gain_near_zero(theta_d,scale=10),4)
        theta_d=round(fth*math.pi,4)
    else:
        theta_d=theta_d
    return round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)

print(Case_5_R(0.3,3))

Factor :  0.84
Ltp:  0.57
vel:  0.727
Tetha sel:  3
Right Wheel vel:  -0.0098
Left Wheel vel:  7.4662
-0.0098


Sixth case: Front sensor nearest, objective angle -

In [131]:
odp=0.1
theta_d=-0.1
if theta_d==0:
    theta_d=-0.001
    
S=-1

factor=round(gain_near_extremes_normalized(odp),4)
print('Factor : ',factor)


Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
print('Ltp: ',Lt_p)
V_p=round(v_max-(factor*(v_mean/2)),4)
print('vel: ',V_p)

if odp<0.2 and theta_d>-0.5 and theta_d<0:
    fth=round(gain_near_zero(-1*theta_d),4)
    print('Factor theta:',fth)
    theta_d=round(fth*-math.pi,4)
else:
    theta_d=theta_d


print('Tetha sel: ',theta_d)

W_R=round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)
W_L=round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)
print('Right Wheel vel: ',W_R)
print('Left Wheel vel: ',W_L)


def Case_6_R(odp,theta_d):
    if theta_d==0:
        theta_d=-0.001
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d>-0.5 and theta_d<0:
        fth=round(gain_near_zero(-1*theta_d),4)
        theta_d=round(fth*-math.pi,4)
    else:
        theta_d=theta_d
    return round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)


def Case_6_L(odp,theta_d):
    if theta_d==0:
        theta_d=-0.001
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d>-0.5 and theta_d<0:
        fth=round(gain_near_zero(-1*theta_d),4)
        theta_d=round(fth*-math.pi,4)
    else:
        theta_d=theta_d
    return round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)

Factor :  0.36
Ltp:  0.39
vel:  0.883
Factor theta: 0.3679
Tetha sel:  -1.1558
Right Wheel vel:  7.0847
Left Wheel vel:  1.9717


### Dataset Creation

In [132]:
Df_C1['Min Value']=Df_C1[['LS','FS','RS']].min(axis=1)
Df_C1.sample(6)


,LS,FS,RS,Thetad,Min Value
2409,0.034902,0.013577,0.426651,0.5675,0.013577
3433,0.637534,0.390453,0.758507,0.1996,0.390453
3313,0.078100,0.068315,0.023163,-0.1509,0.023163
598,0.674439,0.645419,0.284816,1.5485,0.284816
891,0.654556,0.126340,0.673577,-0.4788,0.126340
1284,0.220712,0.974561,0.516157,-2.3221,0.220712


### Left Wheel Evaluation

In [133]:
def lw_eval(df):
    if df['Min Value']== df['LS'] and df['Thetad']>0:
       return Case_1_L(df['Min Value'],df['Thetad']) 
    elif df['Min Value']== df['LS'] and df['Thetad']<0:
       return Case_2_L(df['Min Value'],df['Thetad']) 
    elif df['Min Value']== df['RS'] and df['Thetad']>0:
       return Case_3_L(df['Min Value'],df['Thetad']) 
    elif df['Min Value']== df['RS'] and df['Thetad']<0:
       return Case_4_L(df['Min Value'],df['Thetad']) 
    elif df['Min Value']== df['FS'] and df['Thetad']>0:
       return Case_5_L(df['Min Value'],df['Thetad']) 
    elif df['Min Value']== df['FS'] and df['Thetad']<0:
       return Case_6_L(df['Min Value'],df['Thetad']) 
    
    
    
def rw_eval(df):
      if df['Min Value']== df['LS'] and df['Thetad']>0:
       return Case_1_R(df['Min Value'],df['Thetad']) 
      elif df['Min Value']== df['LS'] and df['Thetad']<0:
       return Case_2_R(df['Min Value'],df['Thetad']) 
      elif df['Min Value']== df['RS'] and df['Thetad']>0:
       return Case_3_R(df['Min Value'],df['Thetad']) 
      elif df['Min Value']== df['RS'] and df['Thetad']<0:
       return Case_4_R(df['Min Value'],df['Thetad']) 
      elif df['Min Value']== df['FS'] and df['Thetad']>0:
       return Case_5_R(df['Min Value'],df['Thetad']) 
      elif df['Min Value']== df['FS'] and df['Thetad']<0:
       return Case_6_R(df['Min Value'],df['Thetad']) 
    

Df_C1=Df_C1.assign(vel_L=Df_C1.apply(lw_eval,axis=1))
Df_C1=Df_C1.assign(vel_R=Df_C1.apply(rw_eval,axis=1))

Df_C1.sample(6)



        

,LS,FS,RS,Thetad,Min Value,vel_L,vel_R
2443,0.753683,0.746463,0.628968,-2.7110,0.628968,1.4422,5.7024
784,0.731226,0.080749,0.778810,-0.6965,0.080749,2.9838,6.2828
3917,0.789301,0.543376,0.938376,0.4305,0.543376,4.2043,2.7434
1446,0.292948,0.684722,0.950386,-0.4304,0.292948,4.1967,3.2976
3380,0.108139,0.088181,0.117958,0.1403,0.088181,6.3245,2.8601
2060,0.372336,0.326997,0.402345,-3.0310,0.326997,0.1041,7.2180


Train Dataframe

In [149]:
Df_C1_Comp=Df_C1[['LS','FS','RS','Thetad','vel_L','vel_R']]
Df_C1_Comp.sample(6)

,LS,FS,RS,Thetad,vel_L,vel_R
2247,0.077242,0.023010,0.205748,-0.5895,3.2352,6.7218
773,0.297668,0.739603,0.090935,0.3341,2.7833,6.3705
3443,0.017594,0.015578,0.140835,-0.3933,4.8392,5.2131
180,0.087255,0.046734,0.097083,-0.0209,-2.0266,11.6892
2937,0.123587,0.229098,0.377752,-2.0280,1.8697,6.9426
3643,0.488485,0.605974,0.023306,2.6080,10.3591,-0.4063


In [150]:
from sklearn.model_selection import train_test_split

train_C1, val_C1 = train_test_split(Df_C1_Comp, test_size=0.2, random_state=42)

Extract vel L  to train and val

In [151]:
train_C1_vl=train_C1[['LS','FS','RS','Thetad','vel_L']]
train_C1_vl.sample(6)

,LS,FS,RS,Thetad,vel_L
125,0.540981,0.490353,0.588361,1.1119,5.3424
338,0.069375,0.199514,0.448670,1.4151,8.1926
2460,0.119010,0.042447,0.102088,0.4579,4.9334
3690,0.980260,0.275404,0.671433,2.5084,8.4514
1321,0.080103,0.133343,0.006291,0.0261,8.9367
3754,0.809755,0.925111,0.059908,-1.7051,0.3905


In [152]:
val_C1_vl=val_C1[['LS','FS','RS','Thetad','vel_L']]
val_C1_vl.sample(6)

,LS,FS,RS,Thetad,vel_L
166,0.154770,0.720166,0.338147,-0.4587,5.6468
3952,0.934916,0.742175,0.347870,-1.9353,1.4416
70,0.661565,0.298414,0.029454,-0.7217,2.8587
1744,0.120727,0.067600,0.547898,-1.9336,-0.0985
2270,0.297525,0.231528,0.743637,2.5917,8.9325
2157,0.855672,0.871945,0.956677,-1.0335,4.1145


Extract vel R, train and val

In [153]:
train_C1_vr=train_C1[['LS','FS','RS','Thetad','vel_R']]
train_C1_vr.sample(6)

,LS,FS,RS,Thetad,vel_R
2068,0.070376,0.101186,0.106806,0.0878,1.4798
828,0.620798,0.148063,0.178010,0.1132,2.1663
2069,0.003862,0.036444,0.117386,-0.4854,2.6656
2833,0.048634,0.967415,0.888762,1.3641,1.1767
624,0.025890,0.035158,0.128253,0.2435,4.1557
493,0.615792,0.842218,0.984844,-2.7088,5.0739


In [154]:
val_C1_vr=val_C1[['LS','FS','RS','Thetad','vel_R']]
val_C1_vr.sample(6)

,LS,FS,RS,Thetad,vel_R
32,0.058218,0.576819,0.712468,1.1179,1.8845
308,0.212559,0.269830,0.088218,-0.7306,6.2763
3318,0.064082,0.131199,0.034029,0.1549,4.3022
436,0.007295,0.017865,0.089362,0.3450,4.7656
3870,0.065513,0.623410,0.909923,2.6505,-1.9185
2566,0.022887,0.037588,0.099657,-0.4552,2.7530


To CSV

In [155]:
import os
parent_dir = 'Train_Val_Data_28'
child_dir_1 = os.path.join(parent_dir, 'Train')
child_dir_2 = os.path.join(parent_dir, 'Validation')
os.makedirs(child_dir_1, exist_ok=True)
os.makedirs(child_dir_2, exist_ok=True)

In [156]:

train_C1_vl.to_csv(os.path.join(child_dir_1, 'C1_vl_train.csv'), index=False)
val_C1_vl.to_csv(os.path.join(child_dir_2, 'C1_vl_val.csv'), index=False)


train_C1_vr.to_csv(os.path.join(child_dir_1, 'C1_vr_train.csv'), index=False)
val_C1_vr.to_csv(os.path.join(child_dir_2, 'C1_vr_val.csv'), index=False)

## No obstacle circle

### Angle Data Generation

In [142]:
HA_Data_2=[round(random.uniform(-math.pi, math.pi),4) for _ in range(3000)]
HA_Data_2=np.array(HA_Data).reshape(3000,1)
print('Angle data :',HA_Data_2[:2])

Angle data : [[-2.2442]
 [-1.3575]]


In [143]:
Df_C2=pd.DataFrame(HA_Data_2,columns=[['HA']])
Df_C2.sample(6)

,HA
30,-0.0089
2853,-1.1080
1371,1.2929
933,-1.8366
1594,1.8971
2875,2.0837


### Evaluation   Right and Left Wheel

In [144]:
Lt_2=1.2
v_max=1

def lw_eval_NO(df):
    Sf1=-1
    haf=df['HA']
    Lt_pf=round((Lt_2),4)
    V_pf=round(v_max,4)
    return round((1-((LMr*haf*Sf1)/(2*Lt_pf)))*(V_pf/R),4)

def rw_eval_NO(df):
    Sf1=-1
    haf=df['HA']
    Lt_pf=round((Lt_2),4)
    V_pf=round(v_max,4)
    return round((1+((LMr*haf*Sf1)/(2*Lt_pf)))*(V_pf/R),4)


Df_C2=Df_C2.assign(W_L=Df_C2.apply(lw_eval_NO,axis=1))

Df_C2=Df_C2.assign(W_R=Df_C2.apply(rw_eval_NO,axis=1))


Df_C2.sample(6)



,HA,W_L,W_R
279,-1.5720,3.8484,6.4080
459,1.7318,6.5381,3.7183
729,2.4896,7.1550,3.1014
2152,-1.2360,4.1220,6.1344
2815,2.8344,7.4357,2.8207
665,-1.2450,4.1146,6.1418


Left Wheel Train NO

In [145]:
Df_C2_Train=Df_C2[['HA','W_L']]
Df_C2_Train.sample(6)

,HA,W_L
872,2.5880,7.2351
1895,0.1196,5.2256
2406,-0.3136,4.8729
398,-1.0793,4.2495
1488,-2.7681,2.8747
2848,0.1757,5.2712


In [146]:
Df_C2_Train.to_csv('Df_C2_Train_L_2.csv',index=False)

Right Wheel Train NO

In [147]:
Df_C2_Train_R=Df_C2[['HA','W_R']]
Df_C2_Train_R.head(6)

,HA,W_R
0,-2.2442,6.9552
1,-1.3575,6.2333
2,3.0486,2.6463
3,1.7736,3.6843
4,1.6729,3.7663
5,0.1402,5.0141


In [148]:
Df_C2_Train_R.to_csv('Df_C2_Train_R_2.csv',index=False)